In [1]:
%load_ext autoreload
%autoreload 2

In [40]:
import os

import wandb
import torch.nn as nn
import numpy as np
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

from IPython.display import clear_output

import matplotlib.pyplot as plt

In [41]:
from datasets.prot.swissprot import SPSetDataset
from methods.protoformer import ProtoFormer

In [4]:
%%script false --no-raise-error


! pip install -r requirements.txt

In [5]:
# %%script false --no-raise-error


# import gdown

# url = 'https://drive.google.com/u/0/uc?id=1a3IFmUMUXBH8trx_VWKZEGteRiotOkZS&export=download'

# if os.path.exists('swissprot.zip'):
#     print('File already downloaded.')
# else:
#     output = 'swissprot.zip'
#     gdown.download(url, output, quiet=False)
#     print('Download completed.')

In [6]:
# %%script false --no-raise-error


# !unzip -q swissprot.zip

# !rm -rf swissprot.zip
# clear_output()

# !mv data/swissprot/go-basic.obo ./

In [22]:
class Id(nn.Module):

    def __init__(self, x_dim):
        super().__init__()
        self.final_feat_dim = x_dim

    def forward(self, x):
        return x

In [8]:
n_way, n_support, n_query, n_train_episode = 5, 5, 15, 5

# Load train dataset. Remember to use make use of functions defined in the `SPSetDataset`.
train_dataset = SPSetDataset(n_way, n_support, n_query, n_episode=n_train_episode, root='./data', mode='train')
train_loader = train_dataset.get_data_loader()

# Load test dataset. Remember to use make use of functions defined in the `SPSetDataset`.
test_dataset = SPSetDataset(n_way, n_support, n_query, n_episode=100, root='./data', mode='test')
test_loader = test_dataset.get_data_loader()


**WARNING: NO VERSION LINE FOUND IN GAF FILE. USING:
!gaf-version: 2.2


ERROR:root:Failed to validate header as GAF v2.2:
[]


HMS:0:00:04.199775 310,057 annotations READ: ./data/swissprot/filtered_goa_uniprot_all_noiea.gaf 
25933 IDs in loaded association branch, BP

**WARNING: NO VERSION LINE FOUND IN GAF FILE. USING:
!gaf-version: 2.2


ERROR:root:Failed to validate header as GAF v2.2:
[]


HMS:0:00:04.265576 310,057 annotations READ: ./data/swissprot/filtered_goa_uniprot_all_noiea.gaf 
25933 IDs in loaded association branch, BP


In [42]:
def train_model(n_way, n_support, n_query, n_train_episode, train_loader, test_loader, backbone):
    wandb.init()
    # Initialize model using the backbone and the optimizer.
    # model = PrototypeFormer(backbone, n_way, n_support, n_sub_supports=n_support-1)
    model = ProtoFormer(backbone, n_way, n_support, n_sub_support=4, n_layer=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

    test_accs = []; train_accs = []; train_losses = []
    for epoch in range(100):
        model.train()

        epoch_loss = model.train_loop(epoch, train_loader, optimizer)
        train_losses.append(epoch_loss)

        train_acc = model.test_loop(train_loader)
        train_accs.append(train_acc)

        test_acc = model.test_loop(test_loader)
        test_accs.append(test_acc)
        print(f'Epoch {epoch} | Train Loss {epoch_loss} | Train Acc {train_acc} | Test Acc {test_acc}')

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 2.5))
    ax1.plot(range(len(train_losses)), train_losses)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Train Loss')
    ax1.grid()

    ax2.plot(range(len(test_accs)), test_accs)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Test Accuracy')
    ax2.grid()
    fig.suptitle(f"n_way={n_way}, n_support={n_support}, n_query={n_query}, n_train_episode={n_train_episode}")

    plt.tight_layout()

In [43]:
os.environ["WANDB_DISABLED"] = "true"
backbone = Id(train_dataset.x_dim)

train_model(n_way, n_support, n_query, n_train_episode, train_loader, test_loader, backbone)

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc650860d30>
Traceback (most recent call last):
  File "/Users/xuyixuan/opt/anaconda3/envs/ML2023/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/Users/xuyixuan/opt/anaconda3/envs/ML2023/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1436, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
Traceback (most recent call last):
  File "<string>", line 1, in <module>
Exception ignored in:   File "/Users/xuyixuan/opt/anaconda3/envs/ML2023/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/xuyixuan/opt/anaconda3/envs/ML2023/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
_p

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)
Epoch 0 | Batch 0/5 | Loss 1.619670
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)
5 Test Acc = 71.73% +- 7.87%
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(relationship)
  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2023-06-11) 46,420 Terms; optional_attrs(rela

KeyboardInterrupt: 